In [1]:
from IPython.display import clear_output
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib notebook 
import numpy as np
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import glob
import h5py as h5
import pandas as pd
import tqdm

/data/keeling/a/jdnied2/miniconda3/envs/Mapping/lib/python3.12/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
font = {
        'size'   : 28
}
matplotlib.rc('font', **font)

# Sample Lat/Lon

In [3]:
files = glob.glob('/data/keeling/a/jdnied2/a/MODIS_ML/data/output/*')

In [4]:
len(files)

1070

In [5]:
def format_bands_to_use(bands_array):
    return [f'Band_{b}.0' for b in bands_array]

In [6]:
def get_number_slices(files):
    retr_size = 0
    for line in tqdm.tqdm(files[:]):
        file = line
        #         key = l_split[1]

        f1 = h5.File(file,'r')
        retr_size += len(f1.keys())
    return retr_size

In [7]:
def get_random_modis_lat_lon(files, retr_size):
    #bands = format_bands_to_use(bands_to_use)
    bands = []
    bands.append('Longitude')
    bands.append('Latitude')
    
    rad_images = np.zeros((retr_size,len(bands),64,64),dtype=np.float32)
    data_index = 0

    for line in tqdm.tqdm(files[:]):
        file = line
#         key = l_split[1]

        f1 = h5.File(file,'r')
        for k in f1.keys():
            group = f1[k]
            labels = np.array(list(group['FeatureLabels'])).astype('U13')

            bdan_indis = []
            for b in bands:
                bdan_indis.append(np.where(labels==b)[0][0])

            rad_images[data_index] = np.array(group['ImageFeatures'])[bdan_indis]


            data_index+=1
        f1.close()

    rad_images_reshaped = np.swapaxes(np.swapaxes(rad_images,2,1),2,3)
    
    return rad_images_reshaped



In [8]:
retr_files = np.random.choice(files, size=300, replace=False)

In [9]:
retr_size = get_number_slices(retr_files)
retr_size

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:59<00:00,  5.02it/s]


24877

In [ ]:
lat_lon_sample = get_random_modis_lat_lon(retr_files, retr_size)

 88%|███████████████████████████████████████████████████████████████████████████████████████████████████              | 263/300 [02:46<00:31,  1.19it/s]

# Map Lat/Lon

In [ ]:
lon_min = np.min(lat_lon_sample[:,:,:,0])
lon_max = np.max(lat_lon_sample[:,:,:,0])
lat_min = np.min(lat_lon_sample[:,:,:,1])
lat_max = np.max(lat_lon_sample[:,:,:,1])

In [ ]:
print(f"Longitude Bounds: {lon_max} {lon_min}")
print(f"Latitude  Bounds: {lat_max} {lat_min}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# m = Basemap(projection='cyl', llcrnrlat=-90, urcrnrlat=90,
#             llcrnrlon=-180, urcrnrlon=180, resolution='c')
m = Basemap(projection='ortho', 
              lat_0=28, lon_0=-50)
m.bluemarble(scale=0.5)

def plot_curved_box_edge(lats, lons):
    x, y = m(lons, lats)
    m.plot(x, y, color='r', linestyle='dashed', linewidth=3.0)

# Generate points for the edges of the box
num_points = 100
lats_bottom = np.linspace(lat_min, lat_min, num_points)
lons_bottom = np.linspace(lon_min, lon_max, num_points)

lats_top = np.linspace(lat_max, lat_max, num_points)
lons_top = np.linspace(lon_min, lon_max, num_points)

lats_left = np.linspace(lat_min, lat_max, num_points)
lons_left = np.linspace(lon_min, lon_min, num_points)

lats_right = np.linspace(lat_min, lat_max, num_points)
lons_right = np.linspace(lon_max, lon_max, num_points)

# Plot the edges of the box
plot_curved_box_edge(lats_bottom, lons_bottom)
plot_curved_box_edge(lats_top, lons_top)
plot_curved_box_edge(lats_left, lons_left)
plot_curved_box_edge(lats_right, lons_right)



# Add a title and show the plot
# plt.show()
plt.title('Terra MODIS & MISR\nTraining/Testing Datasets')
red_patch = mpatches.Patch(color='red', label='Dataset Retrieval')
plt.legend(handles=[red_patch], loc=4)
plt.tight_layout()
plt.savefig('/data/keeling/a/jdnied2/c/MS_THESIS/Support_Figs/Training+Testing_Datasets_Earth.png')

